In [19]:
#import sys
#conda install --yes --prefix {sys.prefix} tksheet
#!pip install tksheet       

In [20]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use("TkAgg")
from matplotlib.backends.backend_tkagg import ( FigureCanvasTkAgg, NavigationToolbar2Tk)
from matplotlib.figure import Figure
import tkinter as tk
from tkinter import ttk
import tksheet

LARGE_FONT= ("Verdana", 12)

data_sec = pd.read_csv('eve1.csv')
data_sec.head()

,Unnamed: 0,timestamp,flow_id,pcap_cnt,event_type,src_ip,src_port,dest_ip,dest_port,proto,...,stats.app_layer.tx.dcerpc_udp,stats.app_layer.tx.dns_udp,stats.app_layer.tx.nfs_udp,stats.app_layer.tx.krb5_udp,stats.app_layer.expectations,stats.http.memuse,stats.http.memcap,stats.ftp.memuse,stats.ftp.memcap,stats.file_store.open_files
0,0,2014-08-01T02:52:29.986487+0200,5.406114e+14,1.0,dns,172.16.165.132,57372.0,172.16.165.2,53.0,UDP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2014-08-01T02:52:30.181417+0200,5.406114e+14,2.0,dns,172.16.165.132,57372.0,172.16.165.2,53.0,UDP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2014-08-01T02:52:30.567661+0200,1.955962e+15,37.0,dns,172.16.165.132,51240.0,172.16.165.2,53.0,UDP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2014-08-01T02:52:30.664068+0200,6.416762e+14,42.0,http,172.16.165.132,49476.0,95.85.17.107,80.0,TCP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2014-08-01T02:52:30.664068+0200,6.416762e+14,42.0,fileinfo,95.85.17.107,80.0,172.16.165.132,49476.0,TCP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Które kolumny nas interesują: timestamp, 
Alert.action, alert.category, alert.signature, alert.severity, alert.metadata.created_at, alert.metadata.updated_at

+ url i hostname: http.hostname, http.url,


 count_alerts - własna miara

pcap_cnt, event_type, src_ip, src_port, dest_ip, dest_port, proto, http.hostname, http.url, http.http_user_agent

alert.metadata.created_at is not null!!!!

Ewentualnie: flow_id, dns.type, dns.id, dns.rrname, dns.rrtype, dns.version, dns.answers, htttp.http_content_type, http.http_method, http.protocol, http.status, fileinfo.sid

In [21]:
data_alerts_only = data_sec.loc[data_sec['alert.metadata.created_at'].notnull(), ['alert.metadata.created_at','alert.category', 'alert.severity', 'http.hostname', 'http.url' ]]


In [22]:
grouped_by_category = data_alerts_only.groupby('alert.category').count()
data_smaller = grouped_by_category[[ "alert.metadata.created_at"]]
#print(data_smaller)

In [23]:
categories = data_smaller.index.tolist()
categories_counts = data_smaller['alert.metadata.created_at'].tolist()

#print(categories)
#print(categories_counts)

In [ ]:

class SecurityApp(tk.Tk):

    def __init__(self, *args, **kwargs):
        tk.Tk.__init__(self, *args, **kwargs)
        tk.Tk.wm_title(self, "Security App")
        
        container = tk.Frame(self)
        container.pack(side="top", fill="both", expand = True)
        container.grid_rowconfigure(0, weight=1)
        container.grid_columnconfigure(0, weight=1)

        self.frames = {}

        for F in (StartPage, PageOne, PageTwo, PageThree, PagePieChart, PageTable):  #tu trzeba dodac każde nowe okienko
            frame = F(container, self)
            self.frames[F] = frame
            frame.grid(row=0, column=0, sticky="nsew")
        self.show_frame(StartPage)

    def show_frame(self, cont):
        frame = self.frames[cont]
        frame.tkraise()

        
class StartPage(tk.Frame):

    def __init__(self, parent, controller):
        tk.Frame.__init__(self,parent)
        label = tk.Label(self, text="Start Page", font=LARGE_FONT)
        label.pack(pady=10,padx=10)

        button = ttk.Button(self, text="Visit Page 1", command=lambda: controller.show_frame(PageOne))
        button.pack()

        button2 = ttk.Button(self, text="Visit Page 2", command=lambda: controller.show_frame(PageTwo))
        button2.pack()

        button3 = ttk.Button(self, text="Graph Page", command=lambda: controller.show_frame(PageThree))
        button3.pack()
        
        button4 = ttk.Button(self, text="Zobacz wykresy", command=lambda: controller.show_frame(PagePieChart))
        button4.pack()
        
        button5 = ttk.Button(self, text="Zobacz tabele", command=lambda: controller.show_frame(PageTable))
        button5.pack()



class PageOne(tk.Frame):

    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        label = tk.Label(self, text="Page One!!!", font=LARGE_FONT)
        label.pack(pady=10,padx=10)

        button1 = ttk.Button(self, text="Back to Home", command=lambda: controller.show_frame(StartPage))
        button1.pack()

        button2 = ttk.Button(self, text="Page Two",command=lambda: controller.show_frame(PageTwo))
        button2.pack()


class PageTwo(tk.Frame):

    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        label = tk.Label(self, text="Page Two!!!", font=LARGE_FONT)
        label.pack(pady=10,padx=10)

        button1 = ttk.Button(self, text="Back to Home", command=lambda: controller.show_frame(StartPage))
        button1.pack()

        button2 = ttk.Button(self, text="Page One", command=lambda: controller.show_frame(PageOne))
        button2.pack()


class PageThree(tk.Frame):

    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        label = tk.Label(self, text="Graph Page!", font=LARGE_FONT)
        label.pack(pady=10,padx=10)

        button1 = ttk.Button(self, text="Back to Home", command=lambda: controller.show_frame(StartPage))
        button1.pack()

        f = Figure(figsize=(5,5), dpi=100)
        a = f.add_subplot(111)
        a.plot([1,2,3,4,5,6,7,8],[5,6,1,3,8,9,3,5])

        canvas = FigureCanvasTkAgg(f, self)
        canvas.draw()
        canvas.get_tk_widget().pack(side=tk.BOTTOM, fill=tk.BOTH, expand=True)

        toolbar = NavigationToolbar2Tk(canvas, self)
        toolbar.update()
        canvas._tkcanvas.pack(side=tk.TOP, fill=tk.BOTH, expand=True)


class PagePieChart(tk.Frame):

    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        label = tk.Label(self, text="Graph Page!", font=LARGE_FONT)
        label.pack(pady=10,padx=10)

        button1 = ttk.Button(self, text="Back to Home", command=lambda: controller.show_frame(StartPage))
        button1.pack()

        labels = categories
        sizes = categories_counts
        explode = (0, 0, 0, 0)

        f, a = plt.subplots()
        a.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%', shadow=False, startangle=90)
        a.axis('equal')  

        canvas = FigureCanvasTkAgg(f, self)
        canvas.draw() 
        canvas.get_tk_widget().pack(side=tk.BOTTOM, fill=tk.BOTH, expand=True)

        toolbar = NavigationToolbar2Tk(canvas, self)  #na dole wykresu
        toolbar.update()
        canvas._tkcanvas.pack(side=tk.TOP, fill=tk.BOTH, expand=True)

        
class PageTable(tk.Frame):

    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        label = tk.Label(self, text="Strona z tabela", font=LARGE_FONT)
        label.pack(pady=10,padx=10)

        button1 = ttk.Button(self, text="Back to Home", command=lambda: controller.show_frame(StartPage))
        button1.pack()

        sheet = tksheet.Sheet(self)
        sheet.pack()
        sheet.set_sheet_data([[f"{ri+cj}" for cj in range(4)] for ri in range(1)])

       

        #dodatkowe funkcje tabeli
        sheet.enable_bindings(("single_select","row_select","column_width_resize","arrowkeys","right_click_popup_menu",
        "rc_select","rc_insert_row","rc_delete_row","copy","cut","paste","delete","undo","edit_cell"))

app = SecurityApp()
app.mainloop()
        
        
